In [ ]:
import cv2 as cv
import numpy as np
from glob import glob
from sklearn.utils import shuffle

from config import efficientNet_config
from models import efficientNetV2B0_model, efficientNetV2B3_model
from utils import plt_heatmap, plt_roccurve

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import os
# 使用第二張 GPU 卡
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# 載入模型

In [ ]:
model = efficientNetV2B0_model()
model.load_weights('../model/202303/20230330.weights')

# 最後一個epoch的模型
# from tensorflow import keras
# model = keras.models.load_model('../model/202303/20230330_lastepoch.h5')

In [ ]:
height  = efficientNet_config['height_B0']
width   = efficientNet_config['width_B0']

In [ ]:
test_dir      = '../dataset_186video_20230323/test'
test_img_arrays = []
test_img_labels = []

## test
img_paths = glob(test_dir + "/0/*.png")
for img_path in img_paths:
    # image
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))     # resize to (224,224)
    test_img_arrays.append(img_array)
    #label
    test_img_labels.append([0])

img_paths = glob(test_dir + "/1/*.png")
for img_path in img_paths:
    # image
    img_array = cv.imread(img_path)
    img_array = cv.resize(img_array,(height,width))      # resize to (224,224)
    test_img_arrays.append(img_array)
    #label
    test_img_labels.append([1])

In [ ]:
test_img_arrays = np.array(test_img_arrays)
test_img_labels = np.array(test_img_labels)

In [ ]:
print("測試集維度= ", test_img_arrays.shape)

# 打亂

In [ ]:
test_img_arrays, test_img_labels = shuffle(test_img_arrays,test_img_labels)

In [ ]:
test_img_arrays.shape

# 預測

In [ ]:
pred_result = model.predict(test_img_arrays)

# 結果

In [ ]:
fpr, tpr, thresholds = roc_curve(test_img_labels, pred_result, pos_label=1) 
roc_auc = auc(fpr, tpr)
plt_roccurve(fpr,tpr,roc_auc)

In [ ]:
pred_result = np.where(pred_result > 0.5, 1,0)
tn, fp, fn, tp = confusion_matrix(test_img_labels, pred_result).ravel()
sensitivity = round(tp / (tp+fn), 4)
specificity = round(tn / (tn+fp), 4)
balanced_acc= round((sensitivity+specificity)/2, 4)
precision   = round(tp / (tp+fp), 4)
f1score     = round(2/((1/precision)+(1/sensitivity)), 4)
accuracy    = round((tp+tn)/(tn+fp+fn+tp), 4)

print('Sensitivity= ',sensitivity)
print('Specificity= ',specificity)
print('Balanced_acc= ',balanced_acc)
print('Precision= ', precision)
print('f1score= ', f1score)
print('Accuracy= ',accuracy)

In [ ]:
plt_heatmap([[tp,fn],[fp,tn]])